In [1]:
import sys
sys.path.append('.')

In [2]:
import datetime
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm

In [3]:
# setting CUDA to load modules lazily
# Only available on CUDA 11.7+
# os.environ['CUDA_MODULE_LOADING'] = 'LAZY'

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision.io import read_image
from torchvision import transforms as T

In [4]:
import time
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter(f"runs/unet_cotnet_{int(time.time())}")

2023-08-04 01:49:38.147342: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-04 01:49:38.467228: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-04 01:49:39.264867: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-04 01:49:39.264913: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [5]:
from change_detection.models.layers import layers
from change_detection.models import models
from change_detection import dataset
from loss.cross_entropy import EdgeCrossEntropy

In [6]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def get_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    return info.used//1024**2


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [7]:
print_gpu_utilization()

GPU memory occupied: 265 MB.


In [8]:
get_gpu_utilization()

265

In [16]:
# # preparing cropped stuff for train, val, and test datasets
ds_path = Path("/home/m/data/projects/dr.eftekhari - change detection/CoTNet/dataset/LEVIR-CD").resolve()
# dataset.make_cropped_dataset(ds_path / "train")
# dataset.make_cropped_dataset(ds_path / "val")
# dataset.make_cropped_dataset(ds_path / "test")

In [17]:
train_ds = dataset.LevirCDDataset(ds_path, limit = 48000, partition = "train")
val_ds = dataset.LevirCDDataset(ds_path, limit = 20000, partition = "val")
test_ds = dataset.LevirCDDataset(ds_path, partition = "test")

In [18]:
len(train_ds), len(test_ds), len(val_ds)

(48000, 25088, 12544)

In [19]:
print_gpu_utilization()

GPU memory occupied: 1407 MB.


In [20]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

# Parameters
params = {'batch_size': 4,
          'shuffle': False,
          'num_workers': 4}
max_epochs = 2
learning_rate = 1e-4

# Dataloaders
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, **params)
test_loader = DataLoader(test_ds, **params)

In [14]:
print_gpu_utilization()

GPU memory occupied: 258 MB.


In [15]:
# Define model
model = models.UnetCotnetNetwork(in_channels=3, classes=2)
if not next(model.parameters()).is_cuda:
    model.to(device)
criterion = EdgeCrossEntropy()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [16]:
print_gpu_utilization()

GPU memory occupied: 1049 MB.


In [17]:
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list)

18115222

In [18]:
class PyTMinMaxScalerVectorized(object):
    """
    Transforms each channel to the range [0, 1].
    """
    def __call__(self, tensor):
        scale = 1.0 / (tensor.max(dim=1, keepdim=True)[0] - tensor.min(dim=1, keepdim=True)[0]) 
        tensor.mul_(scale).sub_(tensor.min(dim=1, keepdim=True)[0])

In [19]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    min_val_loss = np.inf
    for epoch in range(1, n_epochs+1):
        torch.cuda.empty_cache()
        model.train()
        print_gpu_utilization()
        train_loss = 0.0
        for imgs, labels in tqdm(train_loader):
            imgs, labels = imgs.to(device), labels.to(device)
            
            outputs = model(imgs[:, :3, :, :], imgs[:, 3:, :, :])
            loss = loss_fn(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
            torch.cuda.empty_cache()

        # if epoch%3 == 0:
            # validation
        val_loss = 0.0
        model.eval()
        # imgs, labels = imgs.to('cpu'), labels.to('cpu')
        torch.cuda.empty_cache()
        with torch.no_grad():
            for imgs, labels in tqdm(val_loader):
                imgs, labels = imgs.to(device), labels.to(device)

                outputs = model(imgs[:, :3, :, :], imgs[:, 3:, :, :])
                loss = loss_fn(outputs, labels)

                val_loss += loss.item()
        
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        print('=========\n{} Epoch {}\nTraining loss {}\nValidation loss {}'.format(
        datetime.datetime.now(), epoch,
        train_loss,
        val_loss
        ))
        writer.add_scalar("Train loss/epoch", train_loss / len(train_loader), epoch)
        writer.add_scalar("Validation loss/epoch", val_loss / len(val_loader), epoch)
        writer.add_scalar("GPU Utilized/epoch", get_gpu_utilization(), epoch)
        torch.cuda.empty_cache()
        print_gpu_utilization()

        # save each epochs weights
        if min_val_loss > val_loss:
            min_val_loss = val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, f"./unet_cotnet.{epoch}.pth")

In [20]:
training_loop(30, optimizer, model, criterion, train_loader)

GPU memory occupied: 1049 MB.


  0%|                                                                                                                                                                    | 0/6000 [00:00<?, ?it/s]/home/m/.local/lib/python3.10/site-packages/cupy/cuda/compiler.py:464: UserWarning: cupy.cuda.compile_with_cache has been deprecated in CuPy v10, and will be removed in the future. Use cupy.RawModule or cupy.RawKernel instead.
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3136/3136 [01:35<00:00, 32.99it/s]


2023-07-14 04:08:18.032536 Epoch 1
Training loss 0.7547272253731887
Validation loss 0.6542432120031848
GPU memory occupied: 2213 MB.
GPU memory occupied: 2213 MB.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3136/3136 [01:33<00:00, 33.48it/s]


2023-07-14 04:32:46.475508 Epoch 2
Training loss 0.5453564949929715
Validation loss 0.47400997202767403
GPU memory occupied: 2213 MB.
GPU memory occupied: 2213 MB.


...

2023-07-14 11:41:49.051735 Epoch 20
Training loss 0.014038467252627015
Validation loss 0.13759967944660814
GPU memory occupied: 2215 MB.
GPU memory occupied: 2215 MB.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3136/3136 [01:38<00:00, 31.73it/s]


2023-07-14 12:06:15.203484 Epoch 21
Training loss 0.00026890090915064016
Validation loss 0.14797475552946635
GPU memory occupied: 2243 MB.
GPU memory occupied: 2243 MB.


 20%|██████████████████████████████▍                                                                                                                          | 1193/6000 [04:37<18:38,  4.30it/s]


KeyboardInterrupt: 

In [21]:
writer.close()

### Loading Model for inference

In [9]:
def show_outputs(x, is_label=False, figsize=(8,16)):
    fig, axs = plt.subplots(2, x.shape[0], figsize=figsize)
    for i in range(2):
        for j in range(x.shape[0]):
            if is_label:
                axs[i, j].imshow(x[j, i, :, :].detach().cpu().numpy(), cmap="gray")
            else:
                axs[i, j].imshow(x[j, i*3:(i+1)*3, :, :].squeeze(0).detach().cpu().numpy().transpose(1, 2, 0))
                
    plt.axis('off')
    plt.tight_layout()

In [10]:
def handle_output(model, imgs, conf = 0.3):
    outputs = model(imgs[:, :3, :, :], imgs[:, 3:, :, :])
    outputs = torch.nn.functional.sigmoid(outputs)
    outputs = torch.where(outputs > conf, 1.0, 0.0)
    return outputs

In [11]:
torch.cuda.empty_cache()

In [12]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

learning_rate = 1e-4

In [13]:
model = models.UnetCotnetNetwork(in_channels=3, classes=2)
if not next(model.parameters()).is_cuda:
    model.to(device)

model.eval()

criterion = EdgeCrossEntropy()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

checkpoint = torch.load("./unet_cotnet.11.pth")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [44]:
from sklearn.metrics import (
    recall_score,
    f1_score,
    precision_score,
    accuracy_score,
    average_precision_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    PrecisionRecallDisplay,
)

In [45]:
print_gpu_utilization()

GPU memory occupied: 2783 MB.


In [31]:
recall_scores, precision_scores, f1_scores = {"0": [], "1": []}, {"0": [], "1": []}, {"0": [], "1": []}
# ---------------------
with torch.no_grad():
    model.eval()
    for imgs, labels in tqdm(test_loader):
        torch.cuda.empty_cache()
        imgs, labels = imgs.to(device), labels.to(device)
        
        preds = handle_output(model, imgs, conf=0.5)
        for i in range(imgs.shape[0]):
            y_pred_0 = preds[i, 0, :, :].detach().cpu().numpy().ravel().astype(np.uint8)
            y_pred_1 = preds[i, 1, :, :].detach().cpu().numpy().ravel().astype(np.uint8) # real labels
            y_0 = labels[i, 0, :, :].detach().cpu().numpy().ravel().astype(np.uint8)
            y_1 = labels[i, 1, :, :].detach().cpu().numpy().ravel().astype(np.uint8) # real labels
            
            recall_scores["0"].append(recall_score(y_pred_0, y_0, pos_label=0, average="binary", zero_division=1))
            recall_scores["1"].append(recall_score(y_pred_1, y_1, pos_label=1, average="binary", zero_division=1))
            
            precision_scores["0"].append(precision_score(y_pred_0, y_0, pos_label=0, average="binary", zero_division=1))
            precision_scores["1"].append(precision_score(y_pred_1, y_1, pos_label=1, average="binary", zero_division=1))
            
            f1_scores["0"].append(f1_score(y_pred_0, y_0, pos_label=0, average="binary", zero_division=1))
            f1_scores["1"].append(f1_score(y_pred_1, y_1, pos_label=1, average="binary", zero_division=1))


  0%|                                                                 | 0/6272 [00:00<?, ?it/s]


In [47]:
sum(recall_scores["0"]) / len(recall_scores["0"]), sum(recall_scores["1"]) / len(recall_scores["1"])

(0.9046177464264744, 0.7461734693877551)

In [48]:
sum(precision_scores["0"]) / len(precision_scores["0"]), sum(precision_scores["1"]) / len(precision_scores["1"])

(0.9191249035412095, 1.0)

In [49]:
sum(f1_scores["0"]) / len(f1_scores["0"]), sum(f1_scores["1"]) / len(f1_scores["1"])

(0.8607025938418938, 0.7461734693877551)